In [49]:
import keras
import deepchem as dc
import deepchem.models.graph_models as gm
import json
import pandas as pd
import io
import requests
import os


In [28]:
!curl 'https://raw.githubusercontent.com/ATOMScience-org/AMPL/master/atomsci/ddm/examples/tutorials/datasets/Excape_HTR3A.tsv' --output Excape_HTR3A.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  189k  100  189k    0     0   743k      0 --:--:-- --:--:-- --:--:--  742k


In [33]:
orig_df = pd.read_csv('Excape_HTR3A.tsv', sep = '\t', header = 0)
orig_df.head(5)


,Ambit_InchiKey,Original_Entry_ID,Entrez_ID,Activity_Flag,pXC50,DB,Original_Assay_ID,Tax_ID,Gene_Symbol,Ortholog_Group,InChI,SMILES
0,AATPYXMXFBBKFO-KARVIEMNNA-N,CHEMBL526307,3359,N,1.00000,chembl20,565951,9606,HTR3A,2102,InChI=1/C14H17IN4O2/c1-7(2)8-4-11(20-3)9(15)5-...,C=1C(=C(OC2=CN=C(N=C2N)N)C=C(C1OC)I)C(C)C
1,ABIDZKZAFOTHFB-CCUGXXBONA-N,CHEMBL1271790,3359,A,8.17000,chembl20,674333,9606,HTR3A,2102,InChI=1/C21H28N4O3/c1-13-11-27-12-14(2)25(13)2...,C1C2CCN(C1)C[C@H]2NC(C3=C4N=C(OC4=CC=C3)N5[C@H...
2,ABSDJNLAUZQHJY-UHFFFAOYNA-N,CHEMBL2205048,3359,A,7.21000,chembl20,880265,9606,HTR3A,2102,InChI=1/C17H20N2OS/c1-20-14-5-4-6-15(13-14)21-...,N1(CCNCC1)C=2C(=CC=CC2)SC=3C=C(C=CC3)OC
3,ACKJXXOVSOCBPX-PHLAQJRANA-N,11376655,79246,A,8.25181,pubchem,652569,10116,HTR3A,2102,InChI=1/C36H46N6O/c1-26-33(27-14-6-9-17-30(27)...,O=C(NCCCCCCCNC=1C=2CCCCC2N=C3C1C=CC=C3)C4=C(C(...
4,ACKJXXOVSOCBPX-PHLAQJRANA-N,CHEMBL195241,3359,A,8.25000,chembl20,462123,9606,HTR3A,2102,InChI=1/C36H46N6O/c1-26-33(27-14-6-9-17-30(27)...,C=1(N=C2C=CC=CC2=C(C1C)C(=O)NCCCCCCCNC=3C4=C(C...


In [34]:
print(orig_df.shape)

(649, 12)


In [67]:
import atomsci.ddm.utils.curate_data as curate_data
data = orig_df

data['rdkit_smiles'] = data['SMILES'].apply(curate_data.base_smiles_from_smiles)

featurizer=dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)
smiles = data['rdkit_smiles']
data['ConvMol_Featurized'] = featurizer.featurize(smiles)
data.head(5)

,Ambit_InchiKey,Original_Entry_ID,Entrez_ID,Activity_Flag,pXC50,DB,Original_Assay_ID,Tax_ID,Gene_Symbol,Ortholog_Group,InChI,SMILES,rdkit_smiles,ConvMol_Featurized
0,AATPYXMXFBBKFO-KARVIEMNNA-N,CHEMBL526307,3359,N,1.00000,chembl20,565951,9606,HTR3A,2102,InChI=1/C14H17IN4O2/c1-7(2)8-4-11(20-3)9(15)5-...,C=1C(=C(OC2=CN=C(N=C2N)N)C=C(C1OC)I)C(C)C,COc1cc(C(C)C)c(Oc2cnc(N)nc2N)cc1I,<deepchem.feat.mol_graphs.ConvMol object at 0x...
1,ABIDZKZAFOTHFB-CCUGXXBONA-N,CHEMBL1271790,3359,A,8.17000,chembl20,674333,9606,HTR3A,2102,InChI=1/C21H28N4O3/c1-13-11-27-12-14(2)25(13)2...,C1C2CCN(C1)C[C@H]2NC(C3=C4N=C(OC4=CC=C3)N5[C@H...,C[C@H]1COC[C@H](C)N1c1nc2c(C(=O)N[C@@H]3CN4CCC...,<deepchem.feat.mol_graphs.ConvMol object at 0x...
2,ABSDJNLAUZQHJY-UHFFFAOYNA-N,CHEMBL2205048,3359,A,7.21000,chembl20,880265,9606,HTR3A,2102,InChI=1/C17H20N2OS/c1-20-14-5-4-6-15(13-14)21-...,N1(CCNCC1)C=2C(=CC=CC2)SC=3C=C(C=CC3)OC,COc1cccc(Sc2ccccc2N2CCNCC2)c1,<deepchem.feat.mol_graphs.ConvMol object at 0x...
3,ACKJXXOVSOCBPX-PHLAQJRANA-N,11376655,79246,A,8.25181,pubchem,652569,10116,HTR3A,2102,InChI=1/C36H46N6O/c1-26-33(27-14-6-9-17-30(27)...,O=C(NCCCCCCCNC=1C=2CCCCC2N=C3C1C=CC=C3)C4=C(C(...,Cc1c(N2CCN(C)CC2)nc2ccccc2c1C(=O)NCCCCCCCNc1c2...,<deepchem.feat.mol_graphs.ConvMol object at 0x...
4,ACKJXXOVSOCBPX-PHLAQJRANA-N,CHEMBL195241,3359,A,8.25000,chembl20,462123,9606,HTR3A,2102,InChI=1/C36H46N6O/c1-26-33(27-14-6-9-17-30(27)...,C=1(N=C2C=CC=CC2=C(C1C)C(=O)NCCCCCCCNC=3C4=C(C...,Cc1c(N2CCN(C)CC2)nc2ccccc2c1C(=O)NCCCCCCCNc1c2...,<deepchem.feat.mol_graphs.ConvMol object at 0x...


In [109]:
dataset = data
epochs = 20
mode = ''
deterministic = True
pad_batched = False

#gm.GraphConvModel(1).default_generator(dataset, epochs, mode, deterministic, pad_batched)
gm.GraphConvModel(1)

<generator object GraphConvModel.default_generator at 0x7f24f12892e0>